# Introduction

This notebook is a more compact introduction to the advanced analysis using KiMoPack.<br>
It was developed to support a live workshop.

We will go through the following steps:

1. Data Import (here we actually create the data)
2. Data Shaping + Plotting
3. (Optional) comparative analysis
4. Modelling
5. Reporting

In [ ]:
import os,sys
import KiMoPack.plot_func as pf
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
from importlib import reload
reload(pf)
path_to_files = os.sep.join([os.getcwd(), "Data", "Introduction"])
pf.changefonts()
%matplotlib qt

if 1: # if the warnings annoy you.
    import sys
    if not sys.warnoptions:
        import warnings
        warnings.simplefilter("ignore")

FWHM=2.35482

In [ ]:
try:
    #ta=pf.TA("gui") #lazy option
    ta=pf.TA("con_1.SIA",path=path_to_files)
except Exception as e: 
    print('The loading failed, are you using the right format and file location. Check the last cell in this notebook for help.')

If no errors appear, the import was most likely successful, and we can continue with the inspection and shaping of data. <br> 
If there are troubles during or the data does not look like it should, the imported data is stored in the DataFrame **ta.ds_ori**  <br> 
and checking if this looks correct with "ta.ds_ori.head()" is a very good first step for finding the right input parameters.<br>
See the very last cells in this notebook for more help.

## Data Inspection, shaping and RAW plotting
The first step is usually to visually inspect the data. <br> 
In KiMoPack we use three plotting functions for all plotting tasks and three functions for comparative plotting (see Tutorial 3)
``` python
ta.Plot_RAW()
ta.Plot_fit_output()
ta.Plot_Interactive()
```
All plot functions plot in their standard call (as above) multiple plots simultaneously. <br> 
The first argument is a list that calls all the plots that one chooses. For RAW plotting the default is:
``` python
ta.Plot_RAW(range(4))
```

Here we choose to only look at the Matrix, which is plot "0" (see the documentation with "ta.Plot_RAW?" or https://kimopack.readthedocs.io/en/latest/Plotting.html

In [ ]:
%matplotlib qt
ta.bordercut=[400,975]
ta.timelimits=[-0.2,500]
ta.Plot_RAW(0)

And set the interesting wavelength and time points where we want the code to plot Kinetics and spectra respectively. e

During the import these values are set automatically:
``` python
ta.rel_wave = np.arange(300,1000,100)                        #standard
ta.rel_time = [0.2,0.3,0.5,1,3,10,30,100,300,1000,3000,9000] #standard
```
GUI (released soon)

The parameter "wavelength_bin" set the width of the spectral bins. The parameter "time_width_percent" sets a percentual binning for the times.
additional shaping options include rebinning in the spectral range "wave_nm_bin" or in energy scale "equal_energy_bin". See:<br>
https://kimopack.readthedocs.io/en/latest/Plotting.html#plot-shaping-options-without-influence-on-the-fitting and<br>
https://kimopack.readthedocs.io/en/latest/Shaping.html for more details

In [ ]:
ta()

### Create spectra

In [ ]:
def gauss(t,sigma=0.1,mu=0,scale=1):
	y=np.exp(-0.5*((t-mu)**2)/sigma**2)
	y/=sigma*np.sqrt(2*np.pi)
	return y*scale

x=np.arange(300,1100,2)
lines={'GS':(550,50,2),'A':(800,60,1),'B':(600,50,1),'C':(750,70,1)}
df1=pd.concat([pd.DataFrame(gauss(x,sigma=lines[key][1],mu=lines[key][0],scale=lines[key][2]),index=x,columns=[key]) for key in lines.keys()],axis=1)
lines={'GS':(420,40,0.5),'A':(900,40,0.5),'B':(700,40,0.5),'C':(400,60,1)}
df2=pd.concat([pd.DataFrame(gauss(x,sigma=lines[key][1],mu=lines[key][0],scale=lines[key][2]),index=x,columns=[key]) for key in lines.keys()],axis=1)
df=df1+df2
df.plot()

### create kinetics with spectra

Idea: 

* We have defined the spectra for each species (Named A,B,C and the groundstate
* Now we create three distinct experiments where one of the reactions steps **$k_2$** has three different rates
* We assign the previously created spectra to the species and store all in the list ta_list
* The If=0 conditions at the end enables/disables the option to save the generated kinetics and reload them from disk. This shows how you would load the data from disk for your own projects.

In [ ]:
import function_library as func
reload(func)
ta.mod=func.P13

par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/2,vary=True)                  
par.add('k1',value=1/10,vary=True)             
par.add('k2',value=1/4,vary=True)
par.add('k3',value=1/4,vary=True)                    
par.add('t0',value=0,min=-2,max=2,vary=True)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
par.add('explicit_GS')

for key in par.keys():
    par[key].vary=False
ta.par=par

plt.close('all')
ta_list=[]
for a in [1,5,25]:
    ta1=ta.Copy()
    ta1.filename='r2 x %i'%a
    ta1.par['k2'].value=par['k2'].value*a
    ta1.Fit_Global(ext_spectra=df)
    ta1.ds=ta1.re['AC']+ta1.re['AC'].max().max()*np.random.normal(scale=1e-2,size=ta1.re['AC'].shape)
    ta1.factor=a
    ta_list.append(ta1)
    ta1.intensity_range=7e-3
    #ta1.Plot_RAW(0)
if 0:
    for ta in ta_list:
        ta.Save_project()
    ta_list=pf.GUI_open(project_list=['r2 x 1.hdf5','r2 x 25.hdf5','r2 x 5.hdf5'],path=path_to_files)
ta=ta_list[0].Copy()

### Compare the data

In [ ]:
plt.close('all')
ta.Compare_at_time(other=ta_list[1:],rel_time=1)
window=[0.7,1.3,545,555]
ta.Compare_at_time(other=ta_list[1:],rel_time=1,norm_window=window)
ta.Compare_at_time(other=ta_list[1:],rel_time=10,norm_window=window)

In [ ]:
ta.Compare_at_wave(other=ta_list[1:],rel_wave=[900])
window=[0.1,0.2,780,820]
ta.Compare_at_wave(other=ta_list[1:],rel_wave=[900],norm_window=window)

### Fitting a single spectrum

In [ ]:
par=lmfit.Parameters()                                    # create empty parameter object
par.add('k0',value=1/2,vary=True)                  
par.add('k1',value=1/10,vary=True)             
par.add('k2',value=1/4,vary=True)
par.add('k3',value=1/4,vary=True)                    
par.add('t0',value=0,min=-2,max=2,vary=True)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
par.add('explicit_GS')
ta.par=par
ta.mod=func.P13
ta.Fit_Global(ext_spectra=df)
ta.Plot_fit_output()

### Copy paste fit function and make small adjustments

In [ ]:
def P13_split(times,pardf):								
	'''P13 with splitting'''
	c=np.zeros((len(times),4),dtype='float') 						#creation of matrix that will hold the concentrations
	g=gauss(times,sigma=pardf['resolution']/FWHM,mu=pardf['t0']) 	#creating the gaussian pulse that will "excite" our sample
	if 'sub_steps' in list(pardf.index.values):
		sub_steps=pardf['sub_steps']
	else:
		sub_steps=10  													#defining how many extra steps will be taken between the main time_points
	for i in range(1,len(times)):									#iterate over all timepoints
		dc=np.zeros(4,dtype='float')							#the initial change for each concentration, the "3" is representative of how many changes there will be
		dt=(times[i]-times[i-1])/(sub_steps)						# as we are taking smaller steps the time intervals need to be adapted
		c_temp=c[i-1,:]												#temporary matrix holding the changes (needed as we have sub steps and need to check for zero in the end)
		for j in range(int(sub_steps)):
			dc[0]=-pardf['k0']*dt*c_temp[0]-pardf['k2']*pardf['f0']*dt*c_temp[0]+g[i]*dt		
			dc[1]=pardf['k0']*dt*c_temp[0]-pardf['k1']*dt*c_temp[1]
			dc[2]=pardf['k2']*pardf['f0']*dt*c_temp[0]-pardf['k3']*dt*c_temp[2]
			dc[3]=pardf['k1']*dt*c_temp[1]+pardf['k3']*dt*c_temp[2]
			c_temp=c_temp+dc
			c_temp[c_temp<0]=0
		c[i,:] =c_temp												#store the temporary concentrations into the main matrix
	c=pandas.DataFrame(c,index=times)								#write back the right indexes
	c.index.name='time'												#and give it a name
	c.columns=['A','B','C','Inf']									#this is optional but very useful. The species get names that represent some particular states
	if not 'infinite' in list(pardf.index.values):
		c.drop('Inf',axis=1,inplace=True)
	if 'explicit_GS' in list(pardf.index.values):
		c['GS']=-c.sum(axis=1)
	if 'background' in list(pardf.index.values):					#optional but usefull, allow the keyword "background" to be used to fit the background in the global analysis
		c['background']=1											#background always there (flat)
	return c

### Adding the parameters

In [ ]:
df_GS=pd.DataFrame(df.loc[:,'GS'])
df_GS.plot()

In [ ]:
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/2,vary=True)                  
par.add('k1',value=1/40,vary=True)             
par.add('k2',value=1/4,vary=True)
par.add('k3',value=1/4,vary=True)                    
par.add('t0',value=0,min=-2,max=2,vary=True)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
par.add('explicit_GS')
ta.par=par

ta_list=[]
for a in [1,5,25]:
    ta1=ta.Copy()
    ta1.par=par
    ta1.par.add('f0',value=a,vary=True)
    ta1.filename='r2 x %i'%a
    ta_list.append(ta1)

ta1=ta_list[0]
plt.close('all')
if 1: #run now
    ta1.Fit_Global(multi_project=ta_list[1:],unique_parameter=['f0'],same_DAS=True,ext_spectra=df_GS)
    ta1.Save_project(filename='saved_fit_same_DAS.hdf5')
else:  #load saved project
    ta1=pf.TA('saved_fit_same_DAS.hdf5', path=path_to_files)
    
ta1.Plot_fit_output(0,title='combined fitting')


In [ ]:
plt.close('all')
for re in ta1.multi_projects:
    ta1.re=re
    ta1.Plot_fit_output(4)
    ta1.re=ta1.multi_projects[0]

In [ ]:
ta_sep=ta1.Copy()
if 1: #run now
    ta_sep.Fit_Global(multi_project=ta_list[1:],unique_parameter=['f0'],same_DAS=False,ext_spectra=df_GS)
    ta_sep.Save_project(filename='saved_fit_different_DAS.hdf5')
else: #load previous project
    ta_sep=pf.TA('saved_fit_different_DAS.hdf5',path=path_to_files)
ta_sep.Plot_fit_output(0,title='combined fitting different DAS')

In [ ]:
plt.close('all')
for re in ta_sep.multi_projects:
    ta_sep.re=re
    ta_sep.Plot_fit_output(4)
    ta_sep.re=ta_sep.multi_projects[0]

Important, same_DAS does a lot! With same_DAS=True the spectra are coupled (stacked), without each is fitted independently.

There are a number of additional parameter that one should consider
* **weights** (needed for e.g. different pump powers)
* **same_shape_params** (needed if different techniques)


## Let's look on some oscillations

In [ ]:
ta1=pf.TA('con_6.SIA',path=path_to_files)
chirp=[-1.29781491e-11,4.72546618e-08,-6.36421133e-05,3.77396295e-02,-8.08783621e+00]
ta1.Cor_Chirp(fitcoeff=chirp)

ta1.intensity_range=0.005
ta1.log_scale=False
ta1.timelimits=[-0.2,500]
ta1.bordercut=[390,1150]
ta1.scattercut=[525,580]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]
ta1.ignore_time_region=[-0.15,0.1]
ta1.Plot_RAW(0)

In [ ]:
ta1.mod='full_consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.100143,vary=True)                  
par.add('k1',value=1/2.496702,vary=True)             
par.add('k2',value=1/39.963222,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)
par.add('explicit_GS')
ta1.par=par
 
ta1.Fit_Global()
ta1.Plot_fit_output([0,4])

Now we have the main Kinetics and can subtract them. Here simply use the residuals as the next matrix to be fitted and adjust the shaping parameters

In [ ]:
ta2=ta1.Copy()
ta2.ds=ta1.re['AE']

ta2.intensity_range=3e-4
ta2.rel_wave=[620,700,740,800,830,860]
ta2.timelimits=[0.1,10]
ta2.Plot_RAW(0,scale_type='linear')

As an alternative one could have subtracted all (or some) of the contributions using this approach:
``` python
    dicten=pf.Species_Spectra(ta1) # Extract each of the species as a matrix
    ta3=ta1.Copy()                 # Make a copy of the project to test
    ta3.ds=ta1.re['A']-dicten[1]-dicten[2]-dicten['GS'] #subtract one or multiple of the species.
```

Now we load the function file and select a model from it. <br>
Optimizing follows the same procedure

In [ ]:
import function_library as func
ta2.mod=func.oscil_comp   

In [ ]:
par=lmfit.Parameters()                                       # create empty parameter object
par.add('f0',value=1.00561,vary=True)                  
par.add('tk0',value=1/2.8725,vary=True,min=1/4,max=4)
par.add('S0',value=0.975956,vary=True ,min=0,max=1)
ta2.par=par
ta2.ignore_time_region=[-0.15,0.25]
#ta2.Fit_Global(other_optimizers='least_squares')
ta2.Fit_Global()

plt.close('all')
ta2.error_matrix_amplification=1
ta2.Plot_fit_output([0,4],scale_type='linear')

As does calculating the errors

In [ ]:
#This takes about 2min
#ta2.Fit_Global(confidence_level=0.95)

ta2=pf.TA('Fitted_Oscillations_with_confidence.hdf5',path=path_to_files)
ta2.Print_Results()

Finally  we combine the normal model and the oscillation model and make a combined fit.

In [ ]:
reload(func)
ta1.mod=func.manconsec_oscil
ta1.par=ta1.par_fit
for key in ['f0','tk0','S0']:
    ta1.par.add(key,value=ta2.par_fit[key].value)
    ta1.par[key].vary=False
ta1.par['S0'].min=0
ta1.par['S0'].max=1
ta1.Fit_Global()
ta1.Plot_fit_output()

(loading_error)=
# If errors occur during the loading.

The most likely your file format is wrong. There are many options you could sonsider and in general I would like to point to the documentation for opening https://kimopack.readthedocs.io/en/latest/Opening.html

A good option is to inspect files that cause an error when opening. Look for

* separation of numbers ("," or ";" or "\\t")
* if the time is the first number in each column
* if the wavelength are the very first row
* if the decimalplace is separated with a "." or a ","

In [ ]:
filename='con_1.SIA'
path_to_files = os.sep.join([os.getcwd(), "Data", "Introduction"])
print('the files should be here:')
print(path_to_files)

from IPython.display import display, Markdown
with open(path_to_files + os.sep + filename, 'r', encoding='utf-8') as f:
    for i in range(5):  # Change the range for more/less lines
        line = f.readline()
        print(repr(line))